## Library

In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

## Load DataSet

In [3]:
smd = pd.read_csv('../the-movies-dataset/movies_metadata_equal_ratings.csv')
ratings = pd.read_csv('../the-movies-dataset/ratings_equal_movies_metadata.csv')
train = pd.read_csv('../the-movies-dataset/ratings_train.csv')
test = pd.read_csv('../the-movies-dataset/ratings_test.csv')

In [4]:
r_cols = ['user_id','item_id','rating']
ratings_example = pd.read_csv('./ex.dat',sep =' ', names= r_cols, encoding = 'latin-1')

In [5]:
ratings_example

,user_id,item_id,rating
0,0,0,5.0
1,0,1,4.0
2,0,3,2.0
3,0,4,2.0
4,1,0,5.0
5,1,2,4.0
6,1,3,2.0
7,1,4,0.0
8,2,0,2.0
9,2,2,1.0


In [6]:
ratings_example_matrix = ratings_example.values
ratings_example_matrix.shape

(22, 3)

In [13]:
n_test =  np.unique(ratings_example_matrix[:,0]).shape[0]

In [19]:
np.where(ratings_example_matrix[:,1] == 1)[0].astype(np.int32)

array([ 1, 13, 17], dtype=int32)

In [13]:
class CF(object):
    # Y_data is user-item matrix with have 3 column - (user_id, item_id, ratings) 
    # uuCF mode (user-user or item-item)
    # k is number of user similar to the target-user
    def _init_(seft, Y_data, k, dist_func = cosine_similarity, uuCF = 1):
        self.uuCF = uuCF 
        self.Y_data = Y_data
        self.k = k
        self.dist_func = dist_func
        self.Ybar_data = None
        # number of users and items. 
        self.n_users = np.unique(self.Y_data[:,0]).shape[0]
        self.n_items = np.unique(self.Y_data[:,1]).shape[0]
    
    def add(self, new_data):
         """
        Update Y_data matrix when new ratings come.
        For simplicity, suppose that there is no new user or item.
        """
        self.Y_data = np.concatenate((self.Y_data, new_data), axis = 0)
  
    def normalize_Y(self):
        # get all list of user
        users = self.Y_data[:, 0]
        self.Ybar_data = self.Y_data.copy() # save normalized user-item matrix
        # 1 dimention contain list of unique users which
        # after this with contain rating mean value corresponding user_id
        self.means = np.zeros((self.n_user,))
        for n in range(self.n_users):
            # row indices of rating done by user n 
            ids = np.where(users == n)[0].astype(np.int32)
            
            #indices of all item associated with user n
            item_ids = self.Y_data[ids, 1]
            # indices of all ratings associated with user n
            ratings = self.Y_data[ids, 2]
            #take mean
            m = np.mean(ratings)
            if np.isnan(m):
                m = 0 # to avoid empty array and nan value
                
            self.means[n] = m  # save mean value
            self.Ybar_data[ids, 2] = ratings - self.mu[n]

        ################################################
        # form the rating matrix as a sparse matrix. Sparsity is important 
        # for both memory and computing efficiency. For example, if #user = 1M, 
        # #item = 100k, then shape of the rating matrix would be (100k, 1M), 
        # you may not have enough memory to store this. Then, instead, we store 
        # nonzeros only, and, of course, their location
        
        # and sparse matrix which is usually contain non-zero value and index of it
        self.Ybar = sparse.coo_matrix((self.Ybar_data[:,2],(self.Ybar_data[:, 1], self.Ybar_data[:,0])), (self.n_items, self.n_users))
        # Duplicate entries will be summed together.
        # Delete duplicate value
        self.Ybar = self.Ybar.tocsr()
        
    def similarity(self):
        # The "close-enough" factor. lay phan sau gia tri thap phan
        eps = le-6
        self.S = self.dist_func(self.Ybar.T, self.Ybar.T)
    
    def refesh(self):
        """
        Normalize data and calculate similarity matrix again (after
        some few ratings added)
        """
        self.normalize_Y()
        self.similarity()
    
    def fit(self):
        self.refesh()
    
    # u - current user
    # i - current item
    def pred(self, u, i, normalized = 1):
          """ 
        predict the rating of user u for item i (normalized)
        if you need the un
        """
        # Step 1: find all index that contain rating value equal i
        ids = np.where(self.Y_data[:, 1] == i)[0].astype(np.int32)
        # Step 2: find all user that corresponing to list index
        # which mean user that rated item i
        users_rated_i = (self.Y_data[ids, 0].astype(np.int32))
        # Step 3: find similarity btw the current user and  others
        # who already rated i
        sim = self.S[u, users_rated_i]
        # 